# Text Classification

We will use the [distilled version of the BERT base model](https://huggingface.co/distilbert-base-uncased) on a [dataset with news articles](https://huggingface.co/datasets/ag_news) from HuggingFace.

The dataset consists of 120000 training and 7600 testing samples which can be divided into 4 classes: `World` (0), `Sports` (1), `Business` (2), and `Sci/Tech` (3)

In [2]:
!pip install -qq transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.1 MB/s eta 0:00:00


In [3]:
DATASET = 'ag_news'
NUM_LABELS = 4
MODEL = 'distilbert-base-uncased'

Load the dataset with news articles:

In [4]:
from datasets import load_dataset

dataset = load_dataset(DATASET)
dataset

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

Check the format of one sample from our dataset:

In [5]:
dataset['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

Check whether our dataset is balanced (get the number of samples from each class):

In [36]:
import numpy as np

def check_class_balance(class_labels):
  values, counts = np.unique(class_labels, return_counts=True)
  return values, counts

check_class_balance(dataset['train']['label'])

(array([0, 1, 2, 3]), array([30000, 30000, 30000, 30000]))

Load the tokenizer and have a look at it's special tokens:

In [25]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

*What do these tokens mean?*

`[UNK]`, `unk_token` — The unknown token. A token that is not in the vocabulary cannot be converted to an ID and is set to be this token instead.

`[SEP]`, `sep_token` — The separator token, which is used when building a sequence from multiple sequences, e.g. two sequences for sequence classification or for a text and a question for question answering. It is also used as the last token of a sequence built with special tokens.

`[PAD]`, `pad_token` — The token used for padding, for example when batching sequences of different lengths.

`[CLS]`, `cls_token` — The classifier token which is used when doing sequence classification (classification of the whole sequence instead of per-token classification). It is the first token of the sequence when built with special tokens.

`[MASK]`, `mask_token` — The token used for masking values. This is the token used when training this model with masked language modeling. This is the token which the model will try to predict.

Check what exactly does the tokenizer return (when applied on one sample):

In [8]:
first_sample_text = dataset['train'][0]['text']
first_sample_text

"Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."

In [9]:
first_sample_tokens = tokenizer.tokenize(first_sample_text)
first_sample_tokens

['wall',
 'st',
 '.',
 'bears',
 'claw',
 'back',
 'into',
 'the',
 'black',
 '(',
 'reuters',
 ')',
 'reuters',
 '-',
 'short',
 '-',
 'sellers',
 ',',
 'wall',
 'street',
 "'",
 's',
 'd',
 '##wind',
 '##ling',
 '\\',
 'band',
 'of',
 'ultra',
 '-',
 'cy',
 '##nic',
 '##s',
 ',',
 'are',
 'seeing',
 'green',
 'again',
 '.']

In [10]:
first_sample_ids = tokenizer.convert_tokens_to_ids(first_sample_tokens)
first_sample_ids

[2813,
 2358,
 1012,
 6468,
 15020,
 2067,
 2046,
 1996,
 2304,
 1006,
 26665,
 1007,
 26665,
 1011,
 2460,
 1011,
 19041,
 1010,
 2813,
 2395,
 1005,
 1055,
 1040,
 11101,
 2989,
 1032,
 2316,
 1997,
 11087,
 1011,
 22330,
 8713,
 2015,
 1010,
 2024,
 3773,
 2665,
 2153,
 1012]

In [11]:
first_sample_decoded = tokenizer.decode(first_sample_ids)
first_sample_decoded

"wall st. bears claw back into the black ( reuters ) reuters - short - sellers, wall street's dwindling \\ band of ultra - cynics, are seeing green again."

Compare it to what is returned when we use the `preprocess_function`:

In [12]:
def preprocess_function(examples):
  # https://huggingface.co/docs/transformers/pad_truncation
  # truncation=True and padding='max_length' -> pads sequences with [PAD] token to given max sequence length
  return tokenizer(examples['text'], truncation=True, padding='max_length', return_tensors='pt')

first_sample_tokenized = preprocess_function(dataset['train'][0])
first_sample_tokenized

{'input_ids': tensor([[  101,  2813,  2358,  1012,  6468, 15020,  2067,  2046,  1996,  2304,
          1006, 26665,  1007, 26665,  1011,  2460,  1011, 19041,  1010,  2813,
          2395,  1005,  1055,  1040, 11101,  2989,  1032,  2316,  1997, 11087,
          1011, 22330,  8713,  2015,  1010,  2024,  3773,  2665,  2153,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

Preprocess more samples from our dataset at once:

In [13]:
# training on the whole dataset would take more than 5 hours :(
# train_dataset = dataset['train'].map(preprocess_function, batched=True)
# test_dataset = dataset['test'].map(preprocess_function, batched=True)

train_dataset = dataset['train'].shuffle(seed=42).select(range(2500)).map(preprocess_function, batched=True)
test_dataset = dataset['test'].shuffle(seed=42).select(range(500)).map(preprocess_function, batched=True)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [38]:
check_class_balance(train_dataset['label'])

(array([0, 1, 2, 3]), array([625, 640, 576, 659]))

In [39]:
check_class_balance(test_dataset['label'])

(array([0, 1, 2, 3]), array([120, 121, 134, 125]))

Load the model:

In [14]:
from transformers import AutoModelForSequenceClassification

id2label = {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}
label2id = {'World': 0, 'Sports': 1, 'Business': 2, 'Sci/Tech': 3}

model = AutoModelForSequenceClassification.from_pretrained(MODEL,
                                                           num_labels=NUM_LABELS,
                                                           id2label=id2label,
                                                           label2id=label2id)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

Define evaluation metrics and train our model:

In [15]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(p):
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(logits, axis=1)
    return {'accuracy': accuracy_score(p.label_ids, preds)}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    weight_decay=0.0
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.555948,0.842000
2,No log,0.419403,0.876000


TrainOutput(global_step=314, training_loss=0.36695152331309716, metrics={'train_runtime': 243.246, 'train_samples_per_second': 20.555, 'train_steps_per_second': 1.291, 'total_flos': 662360616960000.0, 'train_loss': 0.36695152331309716, 'epoch': 2.0})

Use the trained model to get prediction for some random sentence of your choice using `pipeline`:

https://huggingface.co/docs/transformers/main_classes/pipelines


In [26]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0)
pipe

In [31]:
pipe('This is my business news headline containing business money company words.')

[{'label': 'Business', 'score': 0.8267550468444824}]

In [32]:
pipe('Some player of hockey did something on ice. Goal was made.')

[{'label': 'Sports', 'score': 0.9467237591743469}]

In [34]:
# what happens when we try to predict the label of a sentence that actually
# belongs to a class that wasn't in our data?
pipe('This sentence is about fashion: It is winter so we have to wear sweaters, vests, and trousers.')
# is this correct behaviour?

[{'label': 'Sci/Tech', 'score': 0.8670270442962646}]

*We can notice that because we defined the `id2label` and `label2id` parameters of the model before training it, the model directly returns the label string instead of just the index.*

How can we improve the performance?

- increase dataset size

- **hyperparametr optimization**

In [35]:
lrs = [5e-4, 5e-6]

for lr in lrs:
  model = AutoModelForSequenceClassification.from_pretrained(MODEL,
                                                             num_labels=NUM_LABELS,
                                                             id2label=id2label,
                                                             label2id=label2id)

  training_args = TrainingArguments(
      output_dir='./results',
      num_train_epochs=2,
      per_device_train_batch_size=16,
      evaluation_strategy='epoch',
      learning_rate=lr,
      weight_decay=0.0
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      compute_metrics=compute_metrics
  )

  trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.621433,0.848000
2,No log,0.614066,0.830000


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.725776,0.864000
2,No log,0.576309,0.872000
